### 知识库增强问答

In [10]:
from zhipu import ChatGLM
import os
llm = ChatGLM(api_key=os.environ["ZHIPU_API_KEY"], version ="chatglm_turbo")


In [11]:
llm.invoke("你好")

'你好👋！我是人工智能助手智谱清言，可以叫我小智🤖，很高兴见到你，欢迎问我任何问题。'

### 加载&切分文档

In [12]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=20)

docs = ["docs/langchain_intro.txt", "docs/porsche_intro.txt"]
chunks = []
for doc in docs:
    chunks.extend(TextLoader(doc).load_and_split(splitter))
    
len(chunks)
    



224

### 构建向量索引

In [13]:
from zhipu import ZhipuEmbedding
import os
embedding = ZhipuEmbedding(api_key=os.environ["ZHIPU_API_KEY"])
embd = embedding.embed_query(chunks[-1].page_content)
len(embd)
embd[:4]



1024

[0.0023421330896372045,
 0.03616516046273811,
 0.023525986850004982,
 -0.044469883387348565]

In [14]:
from langchain.vectorstores import FAISS
db =FAISS.from_documents(chunks, embedding=embedding)
db.__dict__

2023-12-21 11:45:23 [INFO][zhipu.py:110]:embedding 224 with self.batch_size=16


100%|██████████| 224/224 [00:14<00:00, 15.37it/s]


{'embedding_function': <zhipu.ZhipuEmbedding at 0x13fc1d720>,
 'index': <faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x13fbe5f80> >,
 'docstore': <langchain_community.docstore.in_memory.InMemoryDocstore at 0x13fbe65f0>,
 'index_to_docstore_id': {0: '5d50c110-d9db-4641-884d-ca7fae2dd5ec',
  1: 'c5f612a0-96ae-4bf4-b022-298f2ec4bd37',
  2: '80622768-ee33-4b8b-8d74-76e3450bedeb',
  3: '2a12866e-a2ea-41f6-a322-b89ad04d2129',
  4: '553a5ca9-d30d-4669-a290-a44320a5ebd7',
  5: '7a1986d8-5502-470c-932a-f407063ec980',
  6: '13cd02ea-fe4e-439e-98b9-d85289c89488',
  7: '5bdc6b2c-71b6-421b-8312-7c5f04d2d494',
  8: '2a0e055e-6bbb-405d-b356-fb889dc462a1',
  9: '4db8c87e-74f9-4d3b-bd37-e5432249f492',
  10: '814c946e-0b06-49c4-a53f-bd5a2e57ecae',
  11: 'b9640189-c1f4-4147-9b8d-2589a6064c84',
  12: 'a9c9ec8f-4d07-465a-8847-2411916dc642',
  13: 'c1764f90-47a7-46d1-a462-15e936e627fd',
  14: 'd30f42d1-a2ac-4080-ad67-0d7e9bf56af7',
  15: 'ed9dd825-dcdc-441a-89ff-66b3

### 知识问答

In [15]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("请根据[相关信息]{context}回答我的问题，如果[相关信息]没有相关的内容，可以不回答，不要编造。问题是:{question}")

In [16]:
from langchain.chains import RetrievalQA
retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",
                                 chain_type_kwargs={"prompt": prompt},
                                 retriever=retriever, return_source_documents=True)

In [17]:
resp = qa.invoke("保时捷和智谱有什么关系？")

In [18]:
print(resp["result"])

根据您提供的参考信息，保时捷和智谱AI的关系并未直接提及。然而，我们可以从保时捷在人工智能和科技领域的投资和合作方面进行一些推测。\n\n保时捷在近年来一直关注新能源汽车、智能网联汽车以及相关技术的发展。保时捷投资了多家科技公司，如电享科技，致力于推动车网互动、光储充一体化等能源数字化领域的发展。此外，保时捷还投资了名为Sensigo的公司，该公司利用人工智能技术升级汽车维修，从诊断到预测问题，以提高客户满意度和降低维修成本。\n\n智谱AI是一家专注于人工智能领域的公司，其主要业务涉及AI技术研发、应用和解决方案。虽然保时捷和智谱AI之间没有直接的投资关系，但两家公司在人工智能和汽车领域有着共同兴趣和合作可能。\n\n综上所述，尽管保时捷和智谱AI之间的具体关系未见明确，但两家公司在人工智能、新能源和汽车科技领域有一定的合作空间。双方可能会在技术创新、智能网联汽车、新能源汽车等方面展开合作，共同推动汽车行业的可持续发展。


In [19]:
print(resp["source_documents"])

[Document(page_content='2023年12月21日，保时捷和智谱AI进行了友好的交流', metadata={'source': 'docs/porsche_intro.txt'}), Document(page_content='保时捷厂家虽然由专业经理人经营，但保时捷家族仍旧拥有制造厂的大部分股权。据统计，该家族所拥有的资产', metadata={'source': 'docs/porsche_intro.txt'}), Document(page_content='1972年\n保时捷改制为股票上市公司，费利·保时捷担任监事会主席，成立保时捷研发中心。\n1973年', metadata={'source': 'docs/porsche_intro.txt'}), Document(page_content='保时捷', metadata={'source': 'docs/porsche_intro.txt'})]
